In [8]:
import numpy as np
import pandas as pd

import poissonlearning as pl
import graphlearning as gl

import storage

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
experiments = storage.load_results(name="mnist", folder="../results")

In [54]:
p_all = list(set((ex["p"] for ex in experiments)))
results_table = pd.DataFrame()
for p in p_all:
    labels_per_class_all = list(set((ex["labels_per_class"][0] for ex in experiments if np.isclose(ex["p"], p))))
    results_p = pd.Series(index=labels_per_class_all, name=p)
    for num_labels in labels_per_class_all:
        selected_experiments = list(
            filter(
                lambda x: 
                    np.isclose(x["p"], p) 
                    and np.all(np.isclose(x["labels_per_class"], num_labels)),
                experiments
            )
        )
        def _compute_accuracy(experiment):
            prob = experiment["solution"].drop(columns=["x", "y", "true_labels"]).to_numpy()
            scores = prob - np.min(prob)
            scores = scores / np.max(scores)

            # Check if scores are similarity or distance
            pred_labels = np.argmax(scores, axis=1)
            accuracy = gl.ssl.ssl_accuracy(experiment["solution"]["true_labels"], pred_labels, sum(experiment["labels_per_class"]))
            return accuracy
        accuracy = np.mean([_compute_accuracy(ex) for ex in selected_experiments])
        results_p[num_labels] = accuracy

    def _extend_results(results_table, new_entries):
        new_index = np.union1d(results_table.index, new_entries.index)
        results_table = results_table.reindex(new_index)
        new_entries = new_entries.reindex(new_index)
        results_table[new_entries.name] = new_entries
        return results_table

    results_table = _extend_results(results_table, results_p)
results_table

/tmp/ipykernel_30522/1103036512.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  results_p = pd.Series(index=labels_per_class_all, name=p)


,2.0,3.0
1,76.252505,NaN
5,92.787879,90.080808
